In [12]:
import numpy as np
from keras import Input, Model
from keras.datasets import mnist
from keras.layers import Conv2D, LeakyReLU, BatchNormalization, Flatten, Dense, Softmax, MaxPool2D, Dropout
from keras.utils import to_categorical
from sklearn.utils import shuffle
from models.VAE import VariationalAutoencoder
from models.VLAE import VLAE

In [3]:
vlae = VLAE(input_dim=(28, 28, 1), log_dir="", kernel_visualization_layer=1, num_samples=1, feature_map_layers=[1], inner_activation="ReLU", decay_rate=0, feature_map_reduction_factor=1, z_dims=[2,2,2])
vlae.load_weights("/Users/leo/Downloads/vlae.h5")

In [4]:
vae = VariationalAutoencoder(input_dim=(128,128,1),encoder_conv_filters=[32, 64, 64, 64],encoder_conv_kernel_size=[3, 3, 3, 3], encoder_conv_strides=[2, 2, 2, 2],decoder_conv_t_filters=[64, 64, 32, 1],decoder_conv_t_kernel_size=[3, 3, 3, 3],decoder_conv_t_strides=[2, 2, 2, 2],z_dims=[2], log_dir="", feature_map_layers=[], kernel_visualization_layer=1, dropout_rate=0.3, use_batch_norm=True, use_dropout=True)
vae.load_weights("/Users/leo/Downloads/vanilla_vae.h5")

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [13]:
x = inpt = Input(shape=(28,28,1))
x = Conv2D(batch_input_shape=(28,28,1), filters=32, kernel_size=3)(x)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Conv2D(filters=64, kernel_size=3)(x)
x = MaxPool2D()(x)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Conv2D(filters=20, kernel_size=3)(x)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Flatten()(x)
x = Dense(100)(x)
x = Dropout(0.25)(x)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dense(10, activation='sigmoid')(x)
x = outpt = Softmax()(x)

model = Model(inpt, outpt)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [27]:
(x_train, y_train), (x_val, y_val) = mnist.load_data()
y_train = to_categorical(y_train)
x_train, y_train = shuffle(x_train, y_train)
x_train = np.expand_dims(x_train, -1)

In [ ]:
model.fit(x_train, y_train, epochs=10)


Epoch 1/10
60000/60000 [==============================] - 160s 3ms/step - loss: 1.5366 - acc: 0.9571

In [18]:
model.save('classifier.h5')

In [ ]:
model.evaluate(np.expand_dims(x_val, -1), to_categorical(y_val))

In [ ]:
num_generations = 10000

In [ ]:
vlae_generations = vlae.decoder.predict([np.random.uniform(-3,3,(num_generations,2)),np.random.uniform(-3,3,(num_generations,2)), np.random.uniform(-3,3,(num_generations,2))]).squeeze()

In [ ]:
vae_generations = vae.decoder.predict(np.random.uniform(-3,3,(num_generations,2))).squeeze()

In [ ]:
model.predict(np.expand_dims(np.array(list(vlae_generations)), -1))

10000/10000 [==============================] - 7s 658us/step


[1.4719620094299317, 0.9883]

In [40]:
num_generations = 10000

(10000, 28, 28)

In [62]:
vlae_generations = vlae.decoder.predict([np.random.uniform(-3,3,(num_generations,2)),np.random.uniform(-3,3,(num_generations,2)), np.random.uniform(-3,3,(num_generations,2))]).squeeze()

In [63]:
vae_generations = vae.decoder.predict(np.random.uniform(-3,3,(num_generations,2))).squeeze()

In [66]:
model.predict(np.expand_dims(np.array(list(vlae_generations)), -1))

array([[0.08507179, 0.23110315, 0.08508088, ..., 0.08828451, 0.08506834,
        0.08506837],
       [0.08507945, 0.23112218, 0.08508925, ..., 0.08819956, 0.08507615,
        0.08507617],
       [0.08507666, 0.23111191, 0.08508645, ..., 0.08823446, 0.08507318,
        0.0850732 ],
       ...,
       [0.08506204, 0.23107728, 0.08507159, ..., 0.08838648, 0.08505865,
        0.08505867],
       [0.08507391, 0.23110564, 0.08508404, ..., 0.08826236, 0.08507059,
        0.08507061],
       [0.08507601, 0.23111221, 0.08508555, ..., 0.08823957, 0.08507262,
        0.08507264]], dtype=float32)